In [1]:
def save_predictions(tracks, frame_number, image_width, image_height):
    # Create a directory to save the text files if it doesn't exist
    save_dir = 'P-3-4'# Save Label Location
    Path(save_dir).mkdir(parents=True, exist_ok=True)

    # Save the predictions to a text file in YOLOv5 format
    save_path = f'{save_dir}/frame_{frame_number}.txt'
    with open(save_path, 'w') as f:
        for track in tracks:
            track_id = track.track_id
            ltrb = track.to_ltrb()
            bbox = ltrb
            class_id = 0    
            # Convert normalized coordinates to absolute pixel values
            x = (bbox[0] + bbox[2]) / (2 * image_width)
            y = (bbox[1] + bbox[3]) / (2 * image_height)
            width = (bbox[2] - bbox[0]) / image_width
            height = (bbox[3] - bbox[1]) / image_height
            if (bbox[3] - bbox[1]) <100 and (bbox[3] - bbox[1])<100 and 0.66<r<1.47:
                line = f"{class_id} {x} {y} {width} {height}\n"
            f.write(line)
def draw_notification(frame):
    height, width, _ = frame.shape
    square_size = min(height, width) // 10
    cv2.rectangle(frame, (width - square_size, 0), (width, square_size), (0, 0, 255), -1)
    return frame
def adjust_contrast(image, alpha, beta):
    """
    Adjust the contrast of the image using the formula: new_image = alpha * image + beta
    """
    adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return adjusted_image

In [2]:
import cv2
import numpy as np
import sys
import glob
from pathlib import Path
import time
import torch



class YoloDetector():

    def __init__(self, model_name):
        
        self.model = self.load_model(model_name)
        self.classes = self.model.names
        #print(self.classes)
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("Using Device: ", self.device)
        

    def load_model(self, model_name):
   
        if model_name:
            model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_name, force_reload=True)
        else:
            model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
        return model

    def score_frame(self, frame):

        self.model.to(self.device)
        downscale_factor = 2
        width = int(frame.shape[1] / downscale_factor)
        height = int(frame.shape[0] / downscale_factor)
        frame = cv2.resize(frame, (width,height))
        #frame = frame.to(self.device)
        
        results = self.model(frame)
        
        labels, cord = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]
        
        return labels, cord

    def class_to_label(self, x):
   
        return self.classes[int(x)]


    def plot_boxes(self, results, frame, height, width, confidence=0.3):
    
        labels, cord = results
        detections = []

        n = len(labels)
        x_shape, y_shape = width, height
        
    
    
        for i in range(n):
            row = cord[i]
            
            if row[4] >= confidence:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                
                if self.class_to_label(labels[i]) == 'drone':
                
                    x_center = x1 + (x2 - x1)
                    y_center = y1 + ((y2 - y1) / 2)
                    
                    tlwh = np.asarray([x1, y1, int(x2-x1), int(y2-y1)], dtype=np.float32)
                    confidence = float(row[4].item())
                    feature = 'drone'
                    
                    detections.append(([x1, y1, int(x2-x1), int(y2-y1)], row[4].item(), 'drone'))
                
        
        return frame, detections

cap = cv2.VideoCapture('validate05.mp4') #Video Input
alpha = 2  # Increase this value to increase contrast
beta = 2     # Increase this value to make the image brighter or decrease to make it darker
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

detectorbackup = YoloDetector(model_name='Yolov5_StrongSORT_OSNet/yolov5/runs/train/exp20/weights/best.pt')

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


from deep_sort_realtime.deepsort_tracker import DeepSort

object_tracker = DeepSort(max_age=100,
                n_init=2,
                nms_max_overlap=1.0,
                max_cosine_distance=0.3,
                nn_budget=None,
                override_track_class=None,
                embedder="mobilenet",
                half=True,
                bgr=True,
                embedder_gpu=True,
                embedder_model_name=None,
                embedder_wts=None,
                polygon=False,
                today=None)
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=True,varThreshold=200,history=50)
frame_number = 0
total_false_positives = 0
new_width = 1920  # New width in pixels
new_height = 1080  # New height in pixels
out = cv2.VideoWriter('M-3-5.mp4', fourcc, 30.0, (new_width, new_height))#Video Output
meanfps=0
while cap.isOpened():

    succes, img = cap.read()
    if not succes:
        break
    #img = cv2.flip(img, 0)
    img = cv2.resize(img, (new_width, new_height))
    img = adjust_contrast(img, alpha, beta)
    fgmask =fgbg.apply(img, 0.0005)
 
    start = time.perf_counter()
    img2 = cv2.cvtColor(fgmask, cv2.COLOR_GRAY2BGR)
    r1=0.66
    r2=1.47
    
    #img, detections = detector.plot_boxes(results, img, height=img.shape[0], width=img.shape[1], confidence=0.4)
    image_height, image_width = img.shape[:2]
    # find the contours of the foreground regions
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # create an empty image for the convex hull
    hull_image = np.zeros_like(img)
    
    # loop through each contour and add its convex hull to the hull image
    for contour in contours:
        hull = cv2.convexHull(contour)
        cv2.drawContours(hull_image, [hull], 0, (0, 255, 0), thickness=1)
    
    # create a mask of the foreground objects
    mask = np.zeros_like(fgmask)
    for contour in contours:
        cv2.drawContours(mask, [contour], 0, 255, -1)
    
    # apply the mask to the original frame
    masked_frame = cv2.bitwise_and(img, img, mask=mask)
    
    # add the convex hull image to the masked frame
    img2 = cv2.addWeighted(masked_frame, 1, hull_image, 1, 0)
    results = detectorbackup.score_frame(img2)
        
    img2, detections = detectorbackup.plot_boxes(tracks, img2, height=img.shape[0], width=img2.shape[1], confidence=0.40)
    tracks = object_tracker.update_tracks(detections, frame=img) # bbs expected to be a list of detections, each in tuples of ( [left,top,w,h], confidence, detection_class )
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        bbox = ltrb
        x1 = int(bbox[0])
        y1 = int(bbox[1])
        x2 = int(bbox[2])
        y2 = int(bbox[3])

        w = x2 - x1
        h = y2 - y1
        if w==0 or h==0:
                r=0
        else:
            r=w/h
        if w <100 and h<100 and r1<r<r2:
            frame_with_notification = draw_notification(img)
            cv2.putText(img, "WARNING!", (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)
            total_false_positives += 1
            save_predictions(results, ltrb, frame_number, image_width, image_height)
            cv2.rectangle(img,(int(bbox[0]), int(bbox[1])),(int(bbox[2]), int(bbox[3])),(0,0,255),2)
            cv2.putText(img, "ID1: " + str(track_id), (int(bbox[0]), int(bbox[1] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
    end = time.perf_counter()
    totalTime = end - start
    fps = 1 / totalTime
    meanfps=meanfps+fps
    frame_number += 1
    cv2.putText(img, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
    out.write(img)
    cv2.imshow('img',cv2.resize(img,None, fx=.5,fy=.5))


    if cv2.waitKey(1) & 0xFF == 27:
        break


# Release and destroy all windows before termination
out.release()
cap.release()
cv2.destroyAllWindows()
print(f'Total False Positives: {total_false_positives}')
meanfps=meanfps/(frame_number-1)
print(f"Average FPS: {meanfps:.2f}")


C:\Users\admin\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\admin/.cache\torch\hub\master.zip
YOLOv5  2023-8-19 Python-3.7.16 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)

Fusing layers... 
Model summary: 267 layers, 46108278 parameters, 0 gradients, 107.6 GFLOPs
Adding AutoShape... 


Using Device:  cuda
Total False Positives: 778
Average FPS: 12.62


In [14]:
meanfps=meanfps/(frame_number-1)
meanfps

0.021716017416033625

In [9]:
ltrb

array([     611.33,      669.53,      634.38,      693.11])

In [9]:
detections

[]

In [9]:
w

2157

In [10]:
h

1577